In [2]:
import sims
import numpy as np
import msprime
import pandas as pd

import useful
import seaborn as sns
import matplotlib
from matplotlib import pyplot as plt
from importlib import reload


import json

In [3]:
# Generation time, mutation rate and recomination rate
RR = 1e-8
MU = 1.29e-8 
GEN_time = 29.0 

# Split Times
T_NEAND_migration = 55000 #time of Neanderthal migration into Out_of_africa population
T_NEAND_AMH = 550000 # split time between AMH and Neanderthal
T_OOF_AF = 65700 # Out_of_Africa migration time
T_NEAND_samples = 38000

# Effective population size
N_ANC = 18500 # N_e of common  AMH and NEanderthal population 
N_ND = 3400 # N_e of Neanderthal
N_AMH = 23000 # N_e of AMH
N_OOF = 1861 # N_e of Out_of_Africa population
N_AF = 27600 # N_e of Africans
N_EU = 13377 #N_e of Europeans

N_EU_bottleneck = 1080
N_EU_growth = 1450
T_EU_growth = 31900
gr_rate = 0.00202
Portion_admix = 0.03

len_sequence = 200000000 # DNA sequence length

n = 250 # number of generated   AF samples
n_neand = 6 #number of generated Neanderthals

rand_sd =1234 #random seed
T = np.array([T_NEAND_migration, T_NEAND_AMH, T_OOF_AF])/GEN_time


N_ND = 3400 # N_e of Neanderthal
N_e = np.array([N_ANC, N_ND, N_AMH, N_OOF, N_AF, N_EU])

n_eu=1
ploidy=2
L=1000
N_ref_pop=250
N_neanderthal=6

In [ ]:
number_chr=5
ts_mas, ND_true_tracts_mas = sims.sim_history_archaic_many_ts(GEN_time, len_sequence, RR, MU, N_e, T,  n, rand_sd, n_neand,  
                              T_NEAND_samples/GEN_time, n_eu, N_EU_growth, 
                              T_EU_growth/GEN_time, N_EU_bottleneck, gr_rate, Portion_admix, ploidy, number_chr)

Neand ancestry:  [0.02947789, 0.025385235]
Neand ancestry:  [0.02621176, 0.02341772]
Neand ancestry:  [0.033959455, 0.032411275]
Neand ancestry:  [0.0213661, 0.02652531]
Neand ancestry:  [0.04064614, 0.042716975]
Done 0
Done 1


In [ ]:
#functions that move the j-th set of tracts by j*len_sequence
def many_in_one(nd_tracts_mas, number_chr):
    nd2=[]
    for _ in range(len(ts_mas)):
        for j in range(len(nd_tracts_mas[_])):
            nd2.append([nd_tracts_mas[_][j][0]+_*len_sequence, nd_tracts_mas[_][j][1]+_*len_sequence])

    return [useful.tracts_eu(nd2, len_sequence*len(ts_mas) ), nd2]

In [ ]:
true_nd=[ND_true_tracts_mas[_][0] for _ in range(len(ts_mas))]  
real_tracts_in_states = many_in_one(true_nd,number_chr)

# DAIseg preparations

In [ ]:
cover_mas=[0.25,0.5, 0.75, 0.999]

for cover in cover_mas:
    for _ in range(len(ts_mas)):
        sims.create_obs_txt_coverage('obs.chr'+str(_+1)+'.cover.'+str(cover)+'.txt', ploidy,  n_eu, ts_mas[_], N_ref_pop, N_neanderthal, n, cover)
    sims.create_bed_smpls_arch_cov('samples.txt', 'test.bed','arch.cover.'+str(cover)+'.txt', len_sequence, cover, 1, n_eu, ploidy)

In [ ]:
! ./run.daiseg.long.sh

In [ ]:
tracts_daiseg_mas=[]
df_daiseg_mas=[]
for cover in cover_mas:
    daiseg=[]
    for _ in range(number_chr):
        daiseg.append(useful.read_out('out.chr'+str(_+1)+'.cover.'+str(cover)+'.archaic.txt')[0])
    tracts_daiseg = many_in_one(daiseg, number_chr)
    tracts_daiseg_mas.append(tracts_daiseg)
    df_daiseg_mas.append(sims.df_result_lonf_chr(real_tracts_in_states, tracts_daiseg, N_neanderthal,  N_ref_pop, 2))

# HMMIX preparations

In [ ]:
#preparation_to_hmmix (ancestral alleles, fasta, vcf)

j_son={"ingroup":['tsk_'+str(_) for _ in range(0, n_eu)], "outgroup" : ['tsk_'+str(_) for _ in range(n_eu, n_eu+N_ref_pop)]}
with open('individuals.json', 'w') as f:
    json.dump(j_son, f)

for _ in range(len(ts_mas)):
    with open('ts.'+'chr'+str(_+1)+'.vcf', "w") as vcf_file:
        ts_mas[_].write_vcf(vcf_file, contig_id=str(_+1))

sims.make_ancestral_fasta('ancestral', ts_mas, len_sequence)

In [ ]:
#then run ./hmmix.sh from hmmix directory

In [ ]:
file='out.tsk_0.hap1.txt'
hmmix_cut_off=[0.75, 0.85, 0.9, 0.95]
hmmix_nd_mas = [sims.read_noND(file, _ , number_chr) for _ in hmmix_cut_off]


#The goal of this cell in to make ONE big chr
tracts_hmmix_mas = [many_in_one(hmmix_nd_mas[_], number_chr) for _ in range(len(hmmix_cut_off))]
df_hmmix_mas = [sims.df_result_lonf_chr(real_tracts_in_states, tracts_hmmix_mas[_], N_neanderthal,  N_ref_pop, 2) for _ in range(len(hmmix_cut_off))]

# Studying of mean length

In [ ]:
bounds=[[0, 10],[10, 20], [20, 30], [30, 40], [40, 60], [60, 120], [120, 200], [200, 5000]]


df=pd.DataFrame(columns=['CHR', 'POS','Marker', 'Length', 'Int_with_real', 'Int_count', '%Truth', 'Length_category'])
tracts=[real_tracts_in_states[1]]+[tracts_hmmix_mas[_][1] for _ in range(len(hmmix_cut_off))]+[tracts_daiseg_mas[_][1] for _ in range(len(cover_mas))]
markers=['real']+['hmmix_'+str(cut) for cut in hmmix_cut_off] + ['daiseg_'+str(cover) for cover in cover_mas]

for _, m in zip(tracts, markers):  
    for j in _:
        #counting number of intersections        
        k=0
        for i in tracts[0]:
            if useful.intersections([j], [i]):
                k+=1
        for b in bounds:            
            if (j[1]-j[0]+1)/1000>=b[0] and (j[1]-j[0]+1)/1000<=b[1]:
                length_cat=b                
        if k==0:
            df.loc[len(df)]=[int(j[1]//len_sequence)+1,j,m, (j[1]-j[0]+1)/1000, 0, k, 0, length_cat]
        else:            
            df.loc[len(df)]=[int(j[1]//len_sequence)+1,j,m, (j[1]-j[0]+1)/1000, 1, k, 
                             useful.len_tracts(useful.intersections([j], tracts[0]))/useful.len_tracts([j]), length_cat]


#proportion of false-positive length    in all length and in class 

for cut in hmmix_cut_off:

    print('The dosage of false positive length in all tract length ', 
          np.array(df[(df['Marker']=='hmmix_'+str(cut)) & (df['Int_count']==0)]['Length']).sum()/np.array(df[(df['Marker']=='hmmix_'+str(cut))]['Length']).sum())



for cover in cover_mas:

    print('The dosage of false positive length in all tract length ', 
          np.array(df[(df['Marker']=='daiseg_'+str(cover)) & (df['Int_count']==0)]['Length']).sum()/np.array(df[(df['Marker']=='daiseg_'+str(cover))]['Length']).sum())

In [ ]:
df

In [ ]:
#deviation of mean by chr

x=[_ for _ in range(number_chr)]
means=[]
for m in markers:
    d=df[df['Marker']==m][['CHR','Length']]    
    means.append(np.array(d.groupby(['CHR']).mean()['Length']))
    

df_means=pd.DataFrame({'CHR':[_ for _ in range(number_chr)], 'real': means[0], 'hmmix_09':means[1], 
                       'hmmix_085':means[2],'daiseg': means[3] })

plt.plot(x, [(x-y)/y for x,y in zip(df_means['hmmix_09'],df_means['real'])], label='hmmix_09')
plt.plot(x, [(x-y)/y for x,y in zip(df_means['hmmix_085'],df_means['real'])], label='hmmix_085')
plt.plot(x, [(x-y)/y for x,y in zip(df_means['daiseg'],df_means['real'])], label='daiseg')


plt.ylabel('Deviation from the mean length divided by the mean')
plt.xlabel('Simulations')
plt.legend()

#plt.savefig('deviation.of.mean.eps', format='eps', transparent=True)

plt.show()


In [ ]:
fig, axes = plt.subplots(3, 3, figsize=(15, 10), sharey=True, sharex=True)
fig.suptitle('Length distribution of tracts')
sns.histplot(ax=axes[0][0], data=df[df['Marker']=='real'], x='Length', alpha=0.6, binwidth=10 ,  kde=True, color='orange')
sns.histplot(ax=axes[0][1], data=df[df['Marker']=='hmmix_0.75'], x='Length', alpha=0.6, binwidth=10,  kde=True, hue='Int_with_real',multiple='stack')
sns.histplot(ax=axes[0][2], data=df[df['Marker']=='hmmix_0.85'], x='Length', alpha=0.6, binwidth=10,  kde=True, hue='Int_with_real',multiple='stack')
sns.histplot(ax=axes[1][0], data=df[df['Marker']=='hmmix_0.9'], x='Length', alpha=0.6, binwidth=10,  kde=True, hue='Int_with_real',multiple='stack')
sns.histplot(ax=axes[1][1], data=df[df['Marker']=='hmmix_0.95'], x='Length', alpha=0.6, binwidth=10,  kde=True, hue='Int_with_real',multiple='stack')
sns.histplot(ax=axes[1][2], data=df[df['Marker']=='daiseg_0.25'], x='Length', alpha=0.6, binwidth=10,  kde=True, hue='Int_with_real',multiple='stack')
sns.histplot(ax=axes[2][0], data=df[df['Marker']=='daiseg_0.5'], x='Length', alpha=0.6, binwidth=10,  kde=True, hue='Int_with_real',multiple='stack')
sns.histplot(ax=axes[2][1], data=df[df['Marker']=='daiseg_0.75'], x='Length', alpha=0.6, binwidth=10,  kde=True, hue='Int_with_real',multiple='stack')
sns.histplot(ax=axes[2][2], data=df[df['Marker']=='daiseg_0.999'], x='Length', alpha=0.6, binwidth=10,  kde=True, hue='Int_with_real',multiple='stack')


axes[0][0].set_title('Real')
axes[0][1].set_title('HMMmix_0.75')
axes[0][2].set_title('HMMmix_0.85')
axes[1][0].set_title('HMMmix_0.9')
axes[1][1].set_title('HMMmix_0.95')
axes[1][2].set_title('DAIseg_0.25')
axes[2][0].set_title('DAIseg_0.5')
axes[2][1].set_title('DAIseg_0.75')
axes[2][2].set_title('DAIseg_0.99')
#plt.savefig('Length distribution.eps', format='eps', transparent=True)
#plt.show()



In [ ]:
k=0
for v in ts_mas[0].variants():
    if k<10:
        print(v.site.ancestral_state, v.site.position, set(v.genotypes[0:502]))
        k+=1
    

In [ ]:
123